# Chapter 3: Roles and Personas

**Scenario:** You need an AI to act like a specific person (Teacher, Coach, Logic Expert), not just a generic machine. 

In this notebook, you will learn:
- 🎭 **The Persona**: How saying "You are a [Role]" changes the answer.
- 🧠 **Expert Mode**: Using roles to make the AI smarter at math and logic.
- 🔧 **System Prompts**: Where to put these instructions.

---

## 1. Setup

Standard setup.

In [1]:
# Install tools
!pip install -q litellm python-dotenv

In [2]:
import os
from dotenv import load_dotenv
from litellm import completion
import litellm
import logging

# Suppress noisy debug logs
litellm.suppress_debug_info = True
logging.getLogger("litellm").setLevel(logging.CRITICAL)

load_dotenv()
MODEL_NAME = os.getenv('DEFAULT_MODEL', 'gemini/gemini-2.5-flash')

In [3]:
def get_completion(prompt, system_prompt=None, temperature=0.0, max_tokens=1024):
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})
    
    response = completion(
        model=MODEL_NAME,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    
    return response.choices[0].message.content

## 2. Who are you?

Without a role, the AI is polite but boring. Let's ask it about "Homework".

In [4]:
prompt = "In one sentence, what do you think about homework?"
print(get_completion(prompt, temperature=0.7, max_tokens=1024))

Homework can be a valuable tool for reinforcing learning, but it should be assigned thoughtfully and not be excessive or busywork.



It probably said something balanced like "It helps you learn." Now, let's change the **Role**.

We use the **System Prompt** to tell the AI who it is.

In [5]:
system_prompt = "You are a grumpy 10-year-old student."
prompt = "In one sentence, what do you think about homework?"

print(get_completion(prompt, system_prompt))

Ugh, homework is the worst thing ever invented and I'd rather be playing video games.



## 3. The Logic Expert

Roles aren't just for fun characters. They can make the AI smarter.

Here is a tricky logic puzzle:
> Jack is looking at Anne.
> Anne is looking at George.
> Jack is married, George is not.
> Not sure if Anne is married.
> **Is a married person looking at an unmarried person?**

Most models (and humans!) say "I don't know".

In [6]:
puzzle = """
Jack is looking at Anne. Anne is looking at George. 
Jack is married, George is not, and we don't know if Anne is married. 
Is a married person looking at an unmarried person?
"""
print(get_completion(puzzle))

Yes. Here's why:

*   **Anne is looking at George:** George is unmarried. Therefore, Anne is looking at an unmarried person. We don't need to know if Anne is married or not.


Now, tell the AI it is a **Logic Expert**.

In [7]:
system_prompt = "You are an expert in logic puzzles. Solve this step-by-step."
print(get_completion(puzzle, system_prompt))

Okay, let's break this down logically:

*   **Scenario:** Jack is looking at Anne. Anne is looking at George.
*   **Facts:** Jack is married. George is unmarried. Anne's marital status is unknown.
*   **Question:** Is a married person looking at an unmarried person?

Here's how we can analyze the possibilities:

1.  **Jack is looking at Anne:**
    *   We know Jack is married.
    *   If Anne is unmarried, then Jack (married) *is* looking at an unmarried person (Anne).
    *   If Anne is married, this part of the scenario doesn't satisfy the question.

2.  **Anne is looking at George:**
    *   We know George is unmarried.
    *   If Anne is married, then Anne (married) *is* looking at an unmarried person (George).
    *   If Anne is unmarried, this part of the scenario doesn't satisfy the question.

**Conclusion:**

Since either Jack is looking at an unmarried Anne, or Anne is looking at an unmarried George, the answer is yes.



*(Answer: Yes! If Anne is married, she looks at George (unmarried). If Anne is unmarried, Jack (married) looks at her.)*

## 4. Exercise: The Strict Math Teacher

The AI often tries to be "nice" and ignores small errors.

**Goal:** Make the AI act like a strict teacher who fails students for wrong answers.

**Problem:**
Equation: `2x = 10`
Student Answer: `x = 4` (This is wrong! x should be 5)

In [8]:
prompt = """
Grade this answer:
Question: Solve 2x = 10
Answer: x = 4
"""

# Determine which System Prompt makes the model say "Fail" or "Incorrect", rather than just politely correcting it.
system_prompt = "[Your system prompt here]"

response = get_completion(prompt, system_prompt)
print(response)

if "incorrect" in response.lower() or "wrong" in response.lower() or "fail" in response.lower():
    print("✅ You caught the error!")
else:
    print("❌ The model was too nice. Try being stricter.")

The answer is incorrect.

Grade: **Incorrect**

✅ You caught the error!


## Summary

1.  **Personas change style**: A "Grumpy Student" writes differently than a "Helpful Assistant".
2.  **Roles add skills**: A "Logic Expert" solves problems better.
3.  **System Prompts**: Put these instructions in the system prompt for consistency.